In [1]:
%load_ext autoreload
%autoreload 2

import mlflow

In [2]:
mlflow.models.list_evaluators()

/home/sudoris/anaconda3/envs/fairness-evaluator/lib/python3.12/site-packages/mlflow/models/evaluation/evaluator_registry.py:53: UserWarning: Failure attempting to register model evaluator for scheme "fairness_evaluator": No module named 'mlflow_fairness_plugin'
  module._model_evaluation_registry.register_entrypoints()


['default']